In [1]:
import sys, os
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../../"))
from scheduler import Scheduler, file_estimated_cost
import pandas as pd
import pickle as pkl
import heapq
import time
import multiprocessing as mp

In [2]:
with open("../queue.pkl", "rb") as fp:
	queue =	pkl.load(fp)
ExtractionTimes = pd.read_csv("../AggregateExtractionTimes/ExtractionTimes.csv", index_col=0)

In [3]:
count = 0
real = 0
empty = 0
for i in range(len(queue)):
	file_item = queue[i]
	best_extractor_index = file_item.best_extractor_index()
	filename = file_item.get_filename()
	
	'''
	if filename in ExtractionTimes.index:
		print("It's in here")
	'''
	try:
		best_time = ExtractionTimes.loc[filename][best_extractor_index]
		if best_extractor_index == 2:
			assert best_time == 5
	except IndexError:
		print(ExtractionTimes[filename])
		print(best_extractor_index)
		break

	if os.path.getsize(filename) == 0:
		empty += 1

	if best_extractor_index == 2 or best_time == 0:
		count += 1
		if os.path.getsize(filename) == 0:
			print(filename)
	else:
		real += 1
print("None: ", count)
print("Real: ", real)
print("Empty: ", empty)

/home/cc/CDIACPub8/Hydrochem_data/.gmtcommands4
/home/cc/CDIACPub8/oceans/David_Starr/README
/home/cc/CDIACPub8/oceans/David_Starr/README~
/home/cc/CDIACPub8/oceans/McArthurII/README
/home/cc/CDIACPub8/oceans/CARINA/S.JohnsonI/README
/home/cc/CDIACPub8/oceans/SOCCOM/096U20160314/Hydrochem_data/.gmtcommands4
/home/cc/CDIACPub8/oceans/ryofumaru10jap/README~
None:  10266
Real:  10103
Empty:  7


In [4]:
def simulate_single_file_extraction(queue):
	best_index = 2 # 2 represents unknown
	extraction_time = 0
	while best_index == 2 or extraction_time == 5: # nonzero extraction time AND not unknown
		file_item = heapq.heappop(queue)
		best_index = file_item.best_extractor_index()
		extraction_time = ExtractionTimes.loc[file_item.get_filename()][file_item.best_extractor_index()]
	time.sleep(extraction_time)
	
	

In [5]:
print(mp.cpu_count())

48


In [10]:
processes=[mp.Process(target=simulate_single_file_extraction, args=(queue,)) for x in range(0, mp.cpu_count())]

for p in processes:
    p.start()

for p in processes:
    p.join()

